# <center> <img src="../img/ITESOLogo.png" alt="ITESO" width="480" height="130"> </center>
# <center> **Departamento de Electrónica, Sistemas e Informática** </center>
---
## <center> **Procesamiento de Datos Masivos** </center>
---
### <center> **Primavera 2025** </center>
---
### <center> **Ejemplos de Spark: Structured Streaming (Kafka consumer application)** </center>

---

**Lab 08**: Structured Streaming with Kafka

**Fecha**:28/03/2025

**Nombre del Estudiante**:David Abraham Naranjo Salgado, Benjamin Leonardo Zarate Solano y Angel David Cortes Pacheco

**Profesor**: Dr. Pablo Camarillo Ramirez

In [1]:
import findspark
findspark.init()

#### Creacion de la conexión con el cluster de spark


In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("SparkSQLStructuredStreaming-Kafka") \
    .master("spark://b053c29da834:7077") \
    .config("spark.ui.port","4040") \
    .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.13:3.5.4") \
    .getOrCreate()
sc = spark.sparkContext

:: loading settings :: url = jar:file:/opt/conda/spark-3.5.4-bin-hadoop3-scala2.13/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
org.apache.spark#spark-sql-kafka-0-10_2.13 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-6ecf8f5b-3c22-459b-b6fd-1a48ea6c5b52;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.13;3.5.4 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.13;3.5.4 in central
	found org.apache.kafka#kafka-clients;3.4.1 in central
	found org.lz4#lz4-java;1.8.0 in central
	found org.xerial.snappy#snappy-java;1.1.10.5 in central
	found org.slf4j#slf4j-api;2.0.7 in central
	found org.apache.hadoop#hadoop-client-runtime;3.3.4 in central
	found org.apache.hadoop#hadoop-client-api;3.3.4 in central
	found commons-logging#commons-logging;1.1.3 in central
	found com.google.code.findbugs#jsr305;3.0.0 in central
	found org.scala-lang.modules#scala-parallel-collections_2.13;1.0.4 in central
	found org.apache.commons#commons-pool2;2.11.1 in centr

### Creación del Kafka Stream

In [3]:
kafka_lines = spark \
                .readStream \
                .format("kafka") \
                .option("kafka.bootstrap.servers", "78dbdddae4ac:9093") \
                .option("subscribe", "kafka-spark-example") \
                .load()

kafka_lines.printSchema()

root
 |-- key: binary (nullable = true)
 |-- value: binary (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestampType: integer (nullable = true)



### Transform binary data into string

In [4]:
kafka_df = kafka_lines.withColumn("value_str", kafka_lines.value.cast("string"))

### Configuración del "Sink" del stream

In [5]:

query = kafka_df \
                .writeStream \
                .outputMode("append") \
                .trigger(processingTime='3 seconds') \
                .format("console") \
                .option("truncate", "false") \
                .start()

query.awaitTermination(300)

25/04/04 15:02:17 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-64b28286-6d6b-48ca-b02b-dc15840a4774. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
25/04/04 15:02:17 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
25/04/04 15:02:17 WARN AdminClientConfig: These configurations '[key.deserializer, value.deserializer, enable.auto.commit, max.poll.records, auto.offset.reset]' were supplied but are not used yet.


-------------------------------------------
Batch: 0
-------------------------------------------
+---+-----+-----+---------+------+---------+-------------+---------+
|key|value|topic|partition|offset|timestamp|timestampType|value_str|
+---+-----+-----+---------+------+---------+-------------+---------+
+---+-----+-----+---------+------+---------+-------------+---------+



-------------------------------------------
Batch: 1
-------------------------------------------
+----+----------------------------------------------------+-------------------+---------+------+-----------------------+-------------+-----------------+
|key |value                                               |topic              |partition|offset|timestamp              |timestampType|value_str        |
+----+----------------------------------------------------+-------------------+---------+------+-----------------------+-------------+-----------------+
|NULL|[48 6F 6C 61 20 54 65 61 6D 42 65 6C 6C 61 63 6F 73]|kafka-spark-example|0        |0     |2025-04-04 15:02:58.786|0            |Hola TeamBellacos|
+----+----------------------------------------------------+-------------------+---------+------+-----------------------+-------------+-----------------+



25/04/04 15:03:03 WARN ProcessingTimeExecutor: Current batch is falling behind. The trigger interval is 3000 milliseconds, but spent 3301 milliseconds


-------------------------------------------
Batch: 2
-------------------------------------------
+----+----------------------------------------+-------------------+---------+------+-----------------------+-------------+-------------+
|key |value                                   |topic              |partition|offset|timestamp              |timestampType|value_str    |
+----+----------------------------------------+-------------------+---------+------+-----------------------+-------------+-------------+
|NULL|[44 61 76 69 64 20 4E 61 72 61 6E 6A 6F]|kafka-spark-example|0        |1     |2025-04-04 15:03:05.235|0            |David Naranjo|
+----+----------------------------------------+-------------------+---------+------+-----------------------+-------------+-------------+

-------------------------------------------
Batch: 3
-------------------------------------------
+----+----------------------------------------+-------------------+---------+------+----------------------+-------------

-------------------------------------------
Batch: 5
-------------------------------------------
+----+-------------------------------------------------+-------------------+---------+------+-----------------------+-------------+----------------+
|key |value                                            |topic              |partition|offset|timestamp              |timestampType|value_str       |
+----+-------------------------------------------------+-------------------+---------+------+-----------------------+-------------+----------------+
|NULL|[50 75 72 6F 20 42 49 44 20 44 41 54 41 21 21 21]|kafka-spark-example|0        |4     |2025-04-04 15:03:32.809|0            |Puro BID DATA!!!|
+----+-------------------------------------------------+-------------------+---------+------+-----------------------+-------------+----------------+



-------------------------------------------
Batch: 6
-------------------------------------------
+----+----------------------------------------------+-------------------+---------+------+-----------------------+-------------+---------------+
|key |value                                         |topic              |partition|offset|timestamp              |timestampType|value_str      |
+----+----------------------------------------------+-------------------+---------+------+-----------------------+-------------+---------------+
|NULL|[4C 61 62 20 30 38 20 50 72 65 73 65 6E 74 65]|kafka-spark-example|0        |5     |2025-04-04 15:03:51.166|0            |Lab 08 Presente|
+----+----------------------------------------------+-------------------+---------+------+-----------------------+-------------+---------------+



False

In [6]:
query.stop()

In [7]:
sc.stop()